In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime

#Visualisation libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from plotly.offline import init_notebook_mode, iplot 
import plotly.graph_objs as go
import plotly.offline as py
!pip install pycountry
import pycountry
py.init_notebook_mode(connected=True)
import folium 
from folium import plugins

# Image
import numpy as np
from PIL import Image

# Animation
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML

# Graphics in retina format 
%config InlineBackend.figure_format = 'retina' 

# Increase the default plot size and set the color scheme
plt.rcParams['figure.figsize'] = 8, 5


# Disable warnings in Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

     |████████████████████████████████| 10.1 MB 4.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=9e7e0faae60f4a312bb2442fae4b01a843d3a6ab9b0f0250fb05680264f11390
  Stored in directory: /root/.cache/pip/wheels/0e/06/e8/7ee176e95ea9a8a8c3b3afcb1869f20adbd42413d4611c6eb4
Successfully built pycountry


In [3]:
df = pd.read_csv('forbes-dataset-2020.csv')
df.head()

,S.NO,Name,Nationality,Current Rank,Previous Year Rank,Sport,Year,earnings ($ million)
0,1,Mike Tyson,USA,1,NaN,boxing,1990,28.6
1,2,Buster Douglas,USA,2,NaN,boxing,1990,26.0
2,3,Sugar Ray Leonard,USA,3,NaN,boxing,1990,13.0
3,4,Ayrton Senna,Brazil,4,NaN,auto racing,1990,10.0
4,5,Alain Prost,France,5,NaN,auto racing,1990,9.0


In [4]:
# Creating a copy of the original dataframe- df
df1 = df.copy()
df1.drop('S.NO',axis=1,inplace=True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  301 non-null    object 
 1   Nationality           301 non-null    object 
 2   Current Rank          301 non-null    int64  
 3   Previous Year Rank    277 non-null    object 
 4   Sport                 301 non-null    object 
 5   Year                  301 non-null    int64  
 6   earnings ($ million)  301 non-null    float64
dtypes: float64(1), int64(2), object(4)
memory usage: 16.6+ KB


In [5]:

# Convert string to datetime64
df1['Year'] = df1['Year'].apply(pd.to_datetime,format='%Y')

#Set Date column as the index column.
df1['year'] = pd.DatetimeIndex(df1['Year']).year  
df1.set_index('year', inplace=True)
df1.drop('Year',axis=1,inplace=True)

# Converting the sport column to uppercase
df1['Sport'] = df1['Sport'].str.upper()
df1.head()

# df is the original dataframe while df1 is a copy where the Date has been set as an index column

,Name,Nationality,Current Rank,Previous Year Rank,Sport,earnings ($ million)
year,,,,,,
1990,Mike Tyson,USA,1,NaN,BOXING,28.6
1990,Buster Douglas,USA,2,NaN,BOXING,26.0
1990,Sugar Ray Leonard,USA,3,NaN,BOXING,13.0
1990,Ayrton Senna,Brazil,4,NaN,AUTO RACING,10.0
1990,Alain Prost,France,5,NaN,AUTO RACING,9.0


1. Deportistas cuyo ranking ha subido al menos dos lugares entre 2010 y 2020.

In [6]:
pd.set_option('display.max_rows', None)

In [7]:
from ast import NameConstant
#Eliminar los que no tienen Ranking y subsetear el DF entre los años 2010 y 2020
df_first = df[ (df["Year"] <= 2020) & (df["Year"] > 2009)].dropna()

df_first = df_first.sort_values(["Name","Year", "Current Rank", "Previous Year Rank"], ascending=[True, True, False, False ])

df_first.drop(index=[226,287,211 ], axis=1, inplace=True)

just_once = df_first["Name"].value_counts().to_frame().reset_index().rename(columns={'index':'Name', 'Name':'count'})
just_once = just_once[ just_once["count"] == 1]
l = just_once["Name"].tolist()

df_first = df_first[ df_first["Name"].isin(l) == False]

names = []


def twoOrMoreRanking(col):
    if (col["Previous Year Rank"]  - col["Current Rank"]) >= 2:
        names.append(col["Name"])
    return names



def cleaningRanks(row):
    if ">" in row["Current Rank"]:
        row["Current Rank"] = 10
    
    if ">" in row["Previous Year Rank"]:
        row["Previous Year Rank"] = 10

    return row

df_first = df_first.astype(str).apply(cleaningRanks, axis=1)


df_first["Current Rank"] = df_first["Current Rank"].astype(int)

df_first["Previous Year Rank"] = df_first["Previous Year Rank"].astype(int)



df_first.apply(twoOrMoreRanking, axis=1)
names = list(set(names))

names


['Phil Mickelson',
 'Manny Pacquiao',
 'Stephen Curry',
 'Kevin Durant',
 'Roger Federer',
 'Floyd Mayweather',
 'Lionel Messi',
 'Kobe Bryant',
 'Tiger Woods',
 'LeBron James',
 'Neymar',
 'Cristiano Ronaldo']

2. Atleta con el menor número de apariciones y mayores ganancias.

In [ ]:
names = df['Name'].value_counts().to_frame().reset_index().rename(columns={'index':'Name', 'Name':'count'})
names
lista = names[names["count"]== 1]["Name"].tolist()  #convertir df a lista
lista
lessnumber = df[df["Name"].isin(lista)].sort_values("earnings ($ million)", ascending = False)
lessnumber


3. Deporte y país con mayor número de atletas no rankeados que entraron en la lista de atletas mejor pagados.

In [ ]:
df1["Previous Year Rank"] = df1["Previous Year Rank"].astype(str).replace('??','NaN').replace('?','NaN').replace('<NA>','NaN').replace('not ranked', 'NaN').replace('none','NaN')

df1

In [ ]:

df_not_ranked = df1[ df1["Previous Year Rank"] == 'NaN' ]

df_not_ranked_grouped_nationality = df_not_ranked.groupby(["Nationality"])["Name"].agg('count').to_frame().reset_index().rename(columns={'Name':'count'}).sort_values('count', ascending=False)
display(df_not_ranked_grouped_nationality)

df_not_ranked_grouped_sport = df_not_ranked.groupby(["Sport"])["Name"].agg('count').to_frame().reset_index().rename(columns={'Name':'count'}).sort_values('count', ascending=False)


display(df_not_ranked_grouped_sport)

In [27]:
df_not_ranked_grouped_sport.values

array([['AMERICAN FOOTBALL', 4],
       ['BOXING', 2],
       ['BASKETBALL', 1],
       ['BOXING', 1],
       ['HOCKEY', 1],
       ['NFL', 1]], dtype=object)

In [28]:
df["Sport"].unique()

array(['boxing', 'auto racing', 'golf', 'basketball', 'Basketball',
       'Boxing', 'Auto Racing', 'Golf', 'Tennis', 'NFL', 'Auto racing',
       'NBA', 'Baseball', 'Ice Hockey', 'American Football / Baseball',
       'tennis', 'ice hockey', 'F1 Motorsports', 'NASCAR', 'Hockey',
       'Auto Racing (Nascar)', 'F1 racing', 'American Football', 'soccer',
       'baseball', 'cycling', 'motorcycle gp', 'Soccer', 'MMA'],
      dtype=object)

4. País con mayor número de deportes con atletas en el dataset.


In [44]:
no_deportes = df1.groupby("Nationality")["Sport"].agg('nunique').to_frame().reset_index().rename(columns={'Sport':'count'}).sort_values('count', ascending=False)
no_deportes


,Nationality,count
21,USA,13
4,Canada,4
9,Germany,3
20,UK,2
3,Brazil,2
13,Northern Ireland,1
19,Switzerland,1
18,Spain,1
17,Serbia,1
16,Russia,1


5. ¿Cuántos atletas por deporte tiene cada país?

In [12]:
cinco = df1.groupby(['Nationality', "Sport"]).Name.nunique()
cinco


Nationality       Sport                       
Argentina         SOCCER                           1
Australia         GOLF                             1
Austria           AUTO RACING                      1
Brazil            AUTO RACING                      1
                  SOCCER                           2
Canada            AUTO RACING                      1
                  BOXING                           1
                  HOCKEY                           1
                  ICE HOCKEY                       1
Dominican         BASEBALL                         1
Filipino          BOXING                           1
Finland           F1 RACING                        1
France            AUTO RACING                      1
Germany           AUTO RACING                      1
                  F1 MOTORSPORTS                   1
                  F1 RACING                        1
Ireland           MMA                              1
Italy             MOTORCYCLE GP                    1

6. Ganancia mínima y máxima (dentro del dataset) por deporte y por país.

In [45]:
df_groupby_sport = df1.groupby("Sport")["earnings ($ million)"].agg("sum").to_frame().reset_index().sort_values("earnings ($ million)", ascending = False)
df_groupby_sport

,Sport,earnings ($ million)
5,BASKETBALL,3418.1
6,BOXING,2298.8
18,SOCCER,2223.5
10,GOLF,2136.5
19,TENNIS,1017.3
0,AMERICAN FOOTBALL,901.2
9,F1 RACING,397.0
2,AUTO RACING,386.3
8,F1 MOTORSPORTS,248.0
4,BASEBALL,146.1


In [46]:
sport_max = df_groupby_sport.iloc[0,:]
sport_min = df_groupby_sport.iloc[-1,:]

In [48]:
sport_max

Sport                   BASKETBALL
earnings ($ million)        3418.1
Name: 5, dtype: object

In [49]:
sport_min

Sport                    NBA
earnings ($ million)    15.2
Name: 16, dtype: object

In [51]:
#ganancia mínima y máxima por país

df_groupby_country = df1.groupby("Nationality")["earnings ($ million)"].agg("sum").to_frame().reset_index().sort_values("earnings ($ million)", ascending = False)

df_groupby_country

,Nationality,earnings ($ million)
21,USA,8786.3
15,Portugal,787.1
19,Switzerland,781.1
0,Argentina,715.5
9,Germany,639.0
20,UK,443.2
3,Brazil,422.0
14,Philippines,242.0
7,Finland,129.0
11,Italy,128.0


In [52]:
country_max = df_groupby_country.iloc[0,:]
country_min = df_groupby_country.iloc[-1,:]

In [53]:
country_max

Nationality                USA
earnings ($ million)    8786.3
Name: 21, dtype: object

In [54]:
country_min

Nationality             Australia
earnings ($ million)          8.5
Name: 1, dtype: object

7. Atleta con mayores ganancias por deporte por década

In [56]:
df_1990_2000 = df[(df["Year"]>= 1990) & (df["Year"]< 2000)]
df_1990_2000

df_1990_2000['Sport'] = df_1990_2000['Sport'].str.upper()

df_1990_2000

,S.NO,Name,Nationality,Current Rank,Previous Year Rank,Sport,Year,earnings ($ million)
0,1,Mike Tyson,USA,1,nan,BOXING,1990,28.6
1,2,Buster Douglas,USA,2,nan,BOXING,1990,26.0
2,3,Sugar Ray Leonard,USA,3,nan,BOXING,1990,13.0
3,4,Ayrton Senna,Brazil,4,nan,AUTO RACING,1990,10.0
4,5,Alain Prost,France,5,nan,AUTO RACING,1990,9.0
5,6,Jack Nicklaus,USA,6,nan,GOLF,1990,8.6
6,7,Greg Norman,Australia,7,nan,GOLF,1990,8.5
7,8,Michael Jordan,USA,8,nan,BASKETBALL,1990,8.1
8,9,Arnold Palmer,USA,8,nan,GOLF,1990,8.1
9,10,Evander Holyfield,USA,8,nan,BOXING,1990,8.1


In [58]:
df_2000_2010 = df[(df["Year"]>= 2000) & (df["Year"]< 2010)]

df_2000_2010['Sport'] = df_2000_2010['Sport'].str.upper()
df_2000_2010


,S.NO,Name,Nationality,Current Rank,Previous Year Rank,Sport,Year,earnings ($ million)
100,101,Michael Schumacher,Germany,1,1,AUTO RACING,2000,59.0
101,102,Tiger Woods,USA,2,2,GOLF,2000,53.0
102,103,Mike Tyson,USA,3,6,BOXING,2000,48.0
103,104,Michael Jordan,USA,4,4,BASKETBALL,2000,37.0
104,105,Grant Hill,USA,5,10,BASKETBALL,2000,26.0
105,106,Dale Earnhardt,USA,6,9,AUTO RACING (NASCAR),2000,24.5
106,107,Shaq O'Neal,USA,7,7,BASKETBALL,2000,24.0
107,108,Lennox Lewis,USA,8,8,BOXING,2000,23.0
108,109,Oscar De La Hoya,USA,9,3,BOXING,2000,23.0
109,110,Kevin Garnett,USA,10,>10,BASKETBALL,2000,21.0


In [59]:
df_2010_2020 = df[(df["Year"]>= 2010) & (df["Year"]< 2020)]

df_2010_2020['Sport'] = df_2010_2020['Sport'].str.upper()

df_2010_2020

,S.NO,Name,Nationality,Current Rank,Previous Year Rank,Sport,Year,earnings ($ million)
191,192,Tiger Woods,USA,1,1,GOLF,2010,105.0
192,193,Floyd Mayweather,USA,2,>10,BOXING,2010,65.0
193,194,Kobe Bryant,USA,3,2,BASKETBALL,2010,48.0
194,195,Phil Mickelson,USA,4,6,GOLF,2010,46.0
195,196,David Beckham,UK,5,5,SOCCER,2010,43.7
196,197,Roger Federer,Switzerland,6,>10,TENNIS,2010,43.0
197,198,LeBron James,USA,7,6,BASKETBALL,2010,42.8
198,199,Manny Pacquiao,Philippines,8,6,BOXING,2010,42.0
199,200,Eli Manning,USA,9,>10,AMERICAN FOOTBALL,2010,39.9
200,201,Terrell Suggs,USA,10,>10,AMERICAN FOOTBALL,2010,38.3


In [60]:
df_groupby_sport_1990_2000 = df_1990_2000.groupby(["Sport","Name"])["earnings ($ million)"].agg("sum").to_frame().reset_index().sort_values(by=['Sport', 'earnings ($ million)'],ascending=False)
df_groupby_sport_1990_2000

,Sport,Name,earnings ($ million)
38,TENNIS,Andre Agassi,53.6
39,TENNIS,Jim Courier,21.6
40,TENNIS,Monica Seles,8.5
37,NFL,Joe Montana,21.0
36,NFL,Emmit Smith,16.5
35,NBA,Shaquille O'Neal,15.2
34,NASCAR,Dale Earnhardt,43.2
32,ICE HOCKEY,Sergei Federov,29.8
33,ICE HOCKEY,Wayne Gretzky,28.0
31,HOCKEY,Joe Sakic,17.9


In [61]:
df_groupby_sport_2000_2010 = df_2000_2010.groupby(["Sport","Name"])["earnings ($ million)"].agg("sum").to_frame().reset_index().sort_values(by=['Sport', 'earnings ($ million)'],ascending=False)
df_groupby_sport_2000_2010 

,Sport,Name,earnings ($ million)
30,TENNIS,Andre Agassi,52.2
31,TENNIS,Roger Federer,35.0
28,SOCCER,David Beckham,185.5
29,SOCCER,Ronaldinho,68.0
27,NASCAR,Dale Earnhardt Jr.,34.0
26,MOTORCYCLE GP,Valentino Rossi,128.0
25,GOLF,Tiger Woods,782.3
24,GOLF,Phil Mickelson,174.0
23,F1 RACING,Michael Schumacher,234.0
22,F1 RACING,Kimi Raikkonen,129.0


In [62]:
df_groupby_sport_2010_2020 = df_2010_2020.groupby(["Sport","Name"])["earnings ($ million)"].agg("sum").to_frame().reset_index().sort_values(by=['Sport', 'earnings ($ million)'],ascending=False)
df_groupby_sport_2010_2020

,Sport,Name,earnings ($ million)
33,TENNIS,Roger Federer,639.8
31,TENNIS,Novak Djokovic,55.8
32,TENNIS,Rafael Nadal,44.5
27,SOCCER,Cristiano Ronaldo,682.1
29,SOCCER,Lionel Messi,611.5
30,SOCCER,Neymar,195.0
28,SOCCER,David Beckham,176.9
26,MMA,Conor McGregor,99.0
25,GOLF,Tiger Woods,429.3
23,GOLF,Phil Mickelson,345.9


8. Ganancia total por cada deporte por cada año.

In [66]:
df["Sport"] = df["Sport"].str.upper()
df_max_earning_year = df.groupby(["Year","Sport"])["earnings ($ million)"].agg("sum").to_frame().reset_index().sort_values(by=['Year', 'earnings ($ million)'],ascending=False)

df_max_earning_year

,Year,Sport,earnings ($ million)
174,2020,SOCCER,304.5
172,2020,BASKETBALL,226.5
171,2020,AMERICAN FOOTBALL,119.6
175,2020,TENNIS,106.3
173,2020,GOLF,62.3
169,2019,SOCCER,341.0
167,2019,BASKETBALL,234.2
166,2019,AMERICAN FOOTBALL,178.8
168,2019,BOXING,94.0
170,2019,TENNIS,93.4


9. Crea un tablero en Power BI que represente los aspectos más importantes del dataset, a tu gusto.

https://datastudio.google.com/s/mU5O3DwVgOs